<a href="https://colab.research.google.com/github/abdulaziz-mukhitdinov/start-here-guidelines/blob/master/deepseek_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-llm-7b-base",
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/584 [00:00<?, ?B/s]

pytorch_model.bin.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [7]:
prompt = "Explain quantum mechanics in simple words."
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [8]:
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
    )

Setting `pad_token_id` to `eos_token_id`:100001 for open-end generation.


In [9]:
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result)

Explain quantum mechanics in simple words.The 19 year old female driver has been identified as 19-year-old Jessica Davis from Hartsville. According to the Aiken County Coroner, Davis was pronounced dead at 1:25 p.m. from what is believed to be a head injury. The crash was reported around 12:00 p.m. on Saturday, August 21st, on the 3300 block of Old Edgefield Road.
Author


In [10]:
!pip install peft

In [11]:
from peft import LoraConfig, get_peft_model

# Configure LoRA
lora_config = LoraConfig(
    r=8,                      # LoRA rank
    lora_alpha=32,            # Scaling factor
    target_modules=["q_proj", "v_proj"],  # Common for transformer attention
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [12]:
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 3,932,160 || all params: 6,914,297,856 || trainable%: 0.0569


In [13]:
train_data = [
    {"prompt": "Question: What is the capital of Germany?\nAnswer:", "answer": " Berlin."},
    {"prompt": "Question: Who wrote Hamlet?\nAnswer:", "answer": " William Shakespeare."},
    {"prompt": "What is the capital of Uzbekistan?", "answer": "Tashkent."},
    {"prompt": "What is the official language of Uzbekistan?", "answer": "Uzbek."},
    {"prompt": "What currency is used in Uzbekistan?", "answer": "Uzbekistani som (UZS)."},
    {"prompt": "Which famous Silk Road city is located in Uzbekistan?", "answer": "Samarkand."},
    {"prompt": "Who was the ruler of the Timurid Empire born in Uzbekistan?", "answer": "Amir Timur (Tamerlane)."},
    {"prompt": "What is the traditional Uzbek flatbread called?", "answer": "Non or lepyoshka."},
    {"prompt": "Which desert covers much of western Uzbekistan?", "answer": "The Kyzylkum Desert."},
    {"prompt": "What is Uzbekistan's Independence Day?", "answer": "September 1 (from the USSR in 1991)."},
    {"prompt": "Which mountain range lies in eastern Uzbekistan?", "answer": "The Tian Shan mountains."},
    {"prompt": "What is the largest river in Uzbekistan?", "answer": "The Amu Darya."},
    {"prompt": "What is the traditional Uzbek rice dish with meat and carrots?", "answer": "Plov (osh)."},
    {"prompt": "Which Uzbek city is known for its medieval Islamic architecture?", "answer": "Bukhara."},
    {"prompt": "What is the name of Uzbekistan's national airline?", "answer": "Uzbekistan Airways."},
    {"prompt": "Which Soviet leader was born in Uzbekistan?", "answer": "Leonid Brezhnev (in Kamensk, now part of Uzbekistan)."},
    {"prompt": "What is the ancient astronomical observatory in Samarkand called?", "answer": "Ulugh Beg Observatory."},
    {"prompt": "Which UNESCO-listed city in Uzbekistan means 'Green City' in Persian?", "answer": "Shakhrisabz."},
    {"prompt": "What is the traditional Uzbek fermented milk drink?", "answer": "Kumys (fermented mare’s milk)."},
    {"prompt": "Which country borders Uzbekistan to the south?", "answer": "Afghanistan."},
    {"prompt": "What is the name of Uzbekistan's parliament?", "answer": "The Oliy Majlis."},
    {"prompt": "Which Uzbek city was a major center of Islamic learning?", "answer": "Bukhara."},
    {"prompt": "What is the traditional Uzbek cloak called?", "answer": "Chapan."},
    {"prompt": "Which lake in Uzbekistan is shrinking due to water diversion?", "answer": "The Aral Sea."},
    {"prompt": "What is the national sport of Uzbekistan?", "answer": "Kurash (traditional wrestling)."},
    {"prompt": "Which Uzbek poet wrote the epic 'Alpomish'?", "answer": "It is a centuries-old oral epic, later transcribed."},
    {"prompt": "What is the traditional Uzbek hand-stitched embroidery called?", "answer": "Suzani."},
    {"prompt": "Which Uzbek city is nicknamed 'City of Winds'?", "answer": "Khiva."},
    {"prompt": "What is the name of Uzbekistan's first satellite?", "answer": "Uzbekiston-1 (launched in 2021)."},
    {"prompt": "Which ancient empire controlled Uzbekistan before the Arabs?", "answer": "The Persian Achaemenid Empire."},
    {"prompt": "What is the traditional Uzbek knife called?", "answer": "Pichok."},
    {"prompt": "Which Uzbek region is famous for its melons?", "answer": "Khorezm."},
    {"prompt": "What is the name of the ancient fortress in Khiva?", "answer": "Ichan-Qala."},
    {"prompt": "Which Uzbek dish features stuffed grape leaves?", "answer": "Dolma."},
    {"prompt": "What is the traditional Uzbek puppet theater called?", "answer": "Qizilqum puppet theater."},
    {"prompt": "Which Uzbek city is home to the Registan Square?", "answer": "Samarkand."},
    {"prompt": "What is the name of Uzbekistan's highest peak?", "answer": "Khazret Sultan (4,643 meters)."},
    {"prompt": "Which Uzbek leader declared independence in 1991?", "answer": "Islam Karimov."},
    {"prompt": "What is the traditional Uzbek cotton robe called?", "answer": "Chapan."},
    {"prompt": "Which ancient Silk Road route passed through Uzbekistan?", "answer": "The Great Silk Road's northern and southern branches."},
    {"prompt": "What is the national flower of Uzbekistan?", "answer": "The cotton flower (unofficially)."},
    {"prompt": "Which Uzbek city was a center of Zoroastrianism?", "answer": "Bukhara."},
    {"prompt": "What is the traditional Uzbek honey-and-nut pastry?", "answer": "Navat."},
    {"prompt": "Which river forms part of Uzbekistan's border with Afghanistan?", "answer": "The Amu Darya."},
    {"prompt": "What is the name of the ancient Uzbek fire-worshipping temples?", "answer": "Zoroastrian fire temples (e.g., in Bukhara)."},
    {"prompt": "Which Uzbek composer wrote the opera 'Layli and Majnun'?", "answer": "Reinhold Glière (with Uzbek libretto)."},
    {"prompt": "What is the traditional Uzbek wedding headdress for brides?", "answer": "Saukele."},
    {"prompt": "Which Soviet republic was Uzbekistan part of?", "answer": "The Uzbek SSR."},
    {"prompt": "What is the name of the ancient bridge in Bukhara?", "answer": "The Taq-i-Zargaron."},
    {"prompt": "Which Uzbek dish is a meat-and-potato stew?", "answer": "Dimlama."},
    {"prompt": "What is the traditional Uzbek ceramic center?", "answer": "Rishtan."},
    {"prompt": "Which Uzbek city was a major stop for Marco Polo?", "answer": "Samarkand."},
]


In [14]:
from torch.utils.data import Dataset, DataLoader

class PromptDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        example = self.data[idx]
        text = example["prompt"] + example["answer"]
        enc = tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=128,
            return_tensors="pt"
        )
        input_ids = enc.input_ids.squeeze()
        attention_mask = enc.attention_mask.squeeze()
        labels = input_ids.clone()
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }

train_dataset = PromptDataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

In [15]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-4)

In [16]:
model.train()

num_epochs = 10

for epoch in range(num_epochs):
    for batch in train_loader:
        batch = {k: v.to(model.device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch + 1} completed.")


Epoch 1 completed.
Epoch 2 completed.
Epoch 3 completed.
Epoch 4 completed.
Epoch 5 completed.
Epoch 6 completed.
Epoch 7 completed.
Epoch 8 completed.
Epoch 9 completed.
Epoch 10 completed.


In [17]:
model.save_pretrained("lora_adapters/")

In [18]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/deepseek-llm-7b-base",
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-llm-7b-base")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
model = PeftModel.from_pretrained(base_model, "lora_adapters/")

In [22]:
def chat(prompt, max_new_tokens=100, temperature=0.7):
    model.eval()
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            pad_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return text


In [21]:
prompt = "Question: Who wrote Hamlet?\nAnswer:"
response = chat(prompt)
print(response)

Question: Who wrote Hamlet?
Answer: William Shakespeare.
Question: What is the capital of France?
Answer: Paris.
Question: What is the tallest mountain in the world?
Answer: Mount Everest.
Question: What is the largest ocean?
Answer: The Pacific Ocean.
Question: What is the highest mountain in Africa?
Answer: Mount Kilimanjaro.
Question: What is the largest desert in the world?
Answer: The Sahara Desert.
Question: What is the most populous continent


In [26]:
# while True:
#     user_input = input("You: ")
#     if user_input.lower() in ["exit", "quit"]:
#         break
#     response = chat(user_input)
#     print("Bot:", response)

from ipywidgets import Text, Output
from IPython.display import display

chat_box = Text(description="You:")
output_box = Output()

def handle_input(sender):
    user_input = chat_box.value
    if user_input.lower() in ["exit", "quit"]:
        chat_box.disabled = True
        print("Chat ended.")
        return
    response = chat(user_input)
    with output_box:
        print("You:", user_input)
        print("Bot:", response)
        print("")
        #print("Bot:", response)
    chat_box.value = ""

chat_box.on_submit(handle_input)
display(chat_box, output_box)


Text(value='', description='You:')

Output()